In [ ]:
import IMAS
import FXP
import IJulia

if true
    # if using Redis via docker
    REDIS_HOST = "localhost"
    REDIS_PORT = 55000
    REDIS_PASSWORD = "redispw"
else
    REDIS_HOST = "redis-19689.c281.us-east-1-2.ec2.cloud.redislabs.com"
    REDIS_PORT = 19689
    REDIS_PASSWORD = ""
end;

In [ ]:
function service_ip_control(session_id, service_name; timeout=10.0)
    client = FXP.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
    
    controller = IMAS.controllers__linear_controller()
    controller.name = "ip"
    
    while true
        # pop inputs
        inputs = FXP.pop!(client, session_id, service_name, :service; timeout, error_on_timeout=false)
        if inputs === nothing
            println("DONE: $(session_id) $(service_name)")
            break
        end
        inputs = NamedTuple(inputs)

        # initialize controller with given PID coefficients
        if ismissing(controller.outputs, :data)
            IMAS.pid_controller(controller, inputs.P, inputs.I, inputs.D)
        end
        # run controller
        control = controller(inputs.setpoint, inputs.value, inputs.time)
        
        # push output
        FXP.push!(client, session_id, service_name, :service; control)
    end

    FXP.disconnect!(client)
end

In [ ]:
# automated spawn
subscription_client = FXP.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
FXP.register_service(subscription_client, "ip_control", service_ip_control)

In [ ]:
# manual spawn
subscription_client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
FXP.register_service(subscription_client, "ip_control", x->println(x))

In [ ]:
service_ip_control("8136311451093097212", "ip_control")